In [56]:
import os
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers,optimizers
import tensorflow as tf
import matplotlib.pyplot as plt

In [57]:
def rotate_270(m):
    N = len(m)
    ret = [[0] * N for _ in range(N)]

    for r in range(N):
        for c in range(N):
            ret[N-1-c][r] = m[r][c]
    return np.array(ret)

def rotate_90(m):
    N = len(m)
    ret = [[0] * N for _ in range(N)]
    # 왜 'ret = [[0] * N] * N'과 같이 하지 않는지 헷갈리시면 연락주세요.

    for r in range(N):
        for c in range(N):
            ret[c][N-1-r] = m[r][c]
    return np.array(ret)
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 1))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48, 1))
        image_array[i] = image
        
    return image_array, image_label



In [58]:
os.getcwd()


'/home/bmllab/bml_pjh/face_data'

In [59]:
os.chdir('/home/bmllab/bml_pjh/face_data')
data=pd.read_csv('fer2013.csv')


In [60]:
#angry와 disgust표정 거의 똑같아 보여서 하나의 표정으로
for i in range(len(data['Usage'])):
    if data.iloc[i,0]==1:
        data.iloc[i,0]=0
    elif data.iloc[i,0]==3:
        data.iloc[i,0]=1
    elif data.iloc[i,0]==4:
        data.iloc[i,0]=2
    elif data.iloc[i,0]==5:
        data.iloc[i,0]=3
    elif data.iloc[i,0]==6:
        data.iloc[i,0]=4

In [61]:
#fear이 오히려 학습에 방해를 줄거같다는 판단하에 drop
data.drop(data.loc[data['emotion']==2].index,axis=0,inplace=True)


In [62]:
train_imgs, train_lbls = parse_data(data[data["Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data["Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data["Usage"] == "PublicTest"])
print(train_imgs.shape)
print(train_lbls.shape)

(19782, 48, 48, 1)
(19782,)


In [63]:
train_imgs=list(train_imgs)
train_lbls=list(train_lbls)
for i in range(len(data[data["Usage"] == "Training"])):
    train_imgs.append(rotate_90(train_imgs[i]))
    train_imgs.append(rotate_270(train_imgs[i]))
    train_lbls.append(train_lbls[i])
    train_lbls.append(train_lbls[i])
    
train_imgs=np.array(train_imgs)
train_lbls=np.array(train_lbls)

In [64]:
train_imgs.shape

(59346, 48, 48, 1)

In [65]:
train_lbls.shape

(59346,)

In [ ]:
train_x=train_imgs/255.0
test_x=test_imgs/255.0
val_x=val_imgs/255
def build_last(input_shape,classes):
    inputs=layers.Input(shape=input_shape)
    
    x=layers.Conv2D(64,(3,3),padding='same')(inputs)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
   
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)

    
    x__=layers.Conv2D(64,(3,3),padding='same')(inputs)
    x__=layers.BatchNormalization()(x__)
    x__=layers.ReLU()(x__)
    x_=tf.add(x,x__)
    x_=layers.Dropout(0.5)(x_)

   
    
    x=layers.Conv2D(128,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
   
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x_=layers.Conv2D(128,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
    x_=layers.Dropout(0.5)(x_)
    
    x=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x=layers.ZeroPadding2D()(x)    
    x=layers.Conv2D(256,(3,3))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)   
    
    x_=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
    
    x=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x=layers.ZeroPadding2D()(x)    
    x=layers.Conv2D(256,(3,3))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)   
    
    x_=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
 
    
    x=layers.GlobalAveragePooling2D()(x_)
    
    pred=layers.Dense(classes,activation="softmax")(x)
    
    model=tf.keras.Model(inputs=inputs,outputs=pred)
    
    return model



def build_last2(input_shape,classes):
    inputs=layers.Input(shape=input_shape)
    
    x=layers.Conv2D(64,(3,3),padding='same')(inputs)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
   
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)

    
    x__=layers.Conv2D(64,(3,3),padding='same')(inputs)
    x__=layers.BatchNormalization()(x__)
    x__=layers.ReLU()(x__)
    x_=tf.add(x,x__)
    x_=layers.Dropout(0.5)(x_)

   
    
    x=layers.Conv2D(128,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
   
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(128,(3,3),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x_=layers.Conv2D(128,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
    x_=layers.Dropout(0.5)(x_)
    
    x=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x=layers.ZeroPadding2D()(x)    
    x=layers.Conv2D(256,(3,3))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)   
    
    x_=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
    x_=layers.Dropout(0.5)(x_)
    
    x=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)    
    
    x=layers.ZeroPadding2D()(x)    
    x=layers.Conv2D(256,(3,3))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)   
    
    x_=layers.Conv2D(256,(3,3),strides=(2,2))(x_)
    x_=layers.BatchNormalization()(x_)
    x_=layers.ReLU()(x_)
    x_=tf.add(x,x_)
 
    
    x=layers.GlobalAveragePooling2D()(x_)
    
    pred=layers.Dense(classes,activation="softmax")(x)
    
    model=tf.keras.Model(inputs=inputs,outputs=pred)
    
    return model



class_names=['angry','happy','sad','surprise','neutral']
print("model build start")
model=build_last(input_shape=(48,48,1), classes=len(class_names))

print("model build complete")
optimizer=optimizers.Adam()
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

batch_size=80
epochs=20

model.fit(train_x,train_lbls,batch_size=batch_size,epochs=epochs,validation_data=(val_x,val_lbls))

os.chdir("/home/bmllab/bml_pjh/face_model")
test_loss,test_acc=model.evaluate(test_x,test_lbls)

print("test_acc: %f"%test_acc)

model.save("emotion_model20210130.h5")




model build start
model build complete
Train on 59346 samples, validate on 2467 samples
Epoch 1/20
59346/59346 [==============================] - 1278s 22ms/sample - loss: 1.1568 - sparse_categorical_accuracy: 0.4930 - val_loss: 1.2828 - val_sparse_categorical_accuracy: 0.4868
Epoch 2/20
59346/59346 [==============================] - 1270s 21ms/sample - loss: 0.8428 - sparse_categorical_accuracy: 0.6576 - val_loss: 0.9267 - val_sparse_categorical_accuracy: 0.6307
Epoch 3/20
 1760/59346 [..............................] - ETA: 20:44 - loss: 0.7001 - sparse_categorical_accuracy: 0.7250